In [0]:
%sh
cd /dbfs/mnt/ereportinterfaz/inseir/
unzip rcc202012.zip



Archive: rcc202012.zip
 inflating: rcc202012.ope

In [0]:
%run /eReport/PkgGeneral/PrcGeneral

In [0]:
%run "/eReport/PkgRcd/Cálculo de Provisiones"

In [0]:
import pandas as pd
from datetime import datetime

class PrcRcdProcesaOrigen():
  
  def __init__(self, id_solicitud):
      self.id_solicitud = id_solicitud
      
      #Obteniendo parametros
      parametros = PrcGeneral(id_solicitud = self.id_solicitud).obtenerParametros()
      self.empresa = parametros['empresa']  
      self.fecha_proceso = parametros['fecha_proceso']  
      self.fecha_proceso_str = self.fecha_proceso.strftime('%Y-%m-%d') 
      
      #Fecha de actualizacion
      fecha_actualizacion = datetime.now()
      self.fecha_actualizacion_str = fecha_actualizacion.strftime("%Y-%m-%d %H:%M:%S")
      
      #Conexion
      connection = PrcGeneral().databaseConnect()
      self.connection = connection
      
      cursor = self.connection.cursor()
      self.cursor = cursor
      
  def CargaClienteSaldos(self):
      select_x ="""SELECT C.ID, A.CLIENTE_PRODUCTO, B.DIAS_ATRASO, A.IMPORTE, ROW_NUMBER() OVER(PARTITION BY A.CLIENTE_PRODUCTO ORDER BY B.DIAS_ATRASO DESC) NUM
                                  FROM CLIENTE_PRODUCTO_CONCEPTO AS A JOIN CLIENTE_PRODUCTO AS B
                                  ON A.CLIENTE_PRODUCTO = B.ID
                                  JOIN CLIENTE_DEUDOR_PRODUCTO CDP
                                  ON CDP.ID = B.CLIENTE_DEUDOR_PRODUCTO
                                  JOIN CLIENTE AS C
                                  ON C.ID = B.CLIENTE
                                  JOIN PROCESO_PRODUCTO S
                                  ON S.PROCESO = 3 -- Indicador de Atraso
                                  AND S.PRODUCTO = CDP.PRODUCTO
                                  JOIN PROCESO_PRODUCTO_DETALLE PPD
                                  ON PPD.PROCESO_PRODUCTO = S.ID
                                  AND PPD.GRUPO_CONCEPTO_SALDO = 1
                                  AND PPD.CONCEPTO_SALDO = A.CONCEPTO_SALDO
                                  AND PPD.SITUACION_CREDITO = B.SITUACION_CREDITO
                                        WHERE C.FECHA_PROCESO = """"'"+ self.fecha_proceso_str +"""'
                                        """
      sql_select_x = pd.read_sql_query(select_x, self.connection)
      df_select_x = pd.DataFrame(sql_select_x)
      
      #Agregamos saldo acumulado
      #df_select_x['SALDO_ACUMULADO'] = df_select_x.groupby('NUM')['IMPORTE'].sum()
      saldo_acu = df_select_x.groupby(['NUM'])['IMPORTE'].sum()
      print(df_select_x)
      print(saldo_acu)
      
      
      #self.cursor.execute(ActualizaIndicadoAtraso)
      #self.connection.commit()
      #print("proceso finalizado")

In [0]:
import pandas as pd
import numpy as np

data = {'Product':['Box','Bottles','Pen','Markers','Bottles','Pen','Markers','Bottles','Box','Markers','Markers','Pen'],
       'State':['Alaska','California','Texas','North Carolina','California','Texas','Alaska','Texas','North Carolina','Alaska','California','Texas'],
       'Sales':[14,24,31,12,13,7,9,31,18,16,18,14]}

df1=pd.DataFrame(data, columns=['Product','State','Sales'])

display(df1)

fd2 = df1.groupby(['State','Product'])['Sales'].agg('sum').reset_index()
display(fd2)


Product,State,Sales
Box,Alaska,14
Bottles,California,24
Pen,Texas,31
Markers,North Carolina,12
Bottles,California,13
Pen,Texas,7
Markers,Alaska,9
Bottles,Texas,31
Box,North Carolina,18
Markers,Alaska,16


State,Product,Sales
Alaska,Box,14
Alaska,Markers,25
California,Bottles,37
California,Markers,18
North Carolina,Box,18
North Carolina,Markers,12
Texas,Bottles,31
Texas,Pen,52
